# finda_appsflyer_sa_prep_20220916

### 디렉토리 설정
- 아래 각 변수에 해당하는 알맞은 경로를 찾아 입력해주세요.
- 경로 주소는 '.../.../...' 형식을 따라주세요.

In [ ]:
class directory:
    dropbox_dir = 'C:/Dropbox (주식회사매드업)'
    # 드롭박스 폴더 위치경로
    
    raw_dir = dropbox_dir + ''
    # raw_data가 적재되어 있는 폴더 위치경로
    
    download_dir = 'C:/Users/MADUP/Downloads'
    # 최종 결과파일을 다운받을 폴더 위치경로

dr = directory()

### 날짜 설정
- 보통 작일자 기준으로 요청주신 기간의 데이터를 출력하게 됩니다.
- 날짜 조정이 필요한 경우 아래 변수 값을 수정하여 출력해주세요.

In [ ]:
import datetime

class report_date:
    today = datetime.date.today()
    # 오늘 날짜 ex> 2022-09-14
    
    # today = datetime.date(year = 2022, month = 8, day = 12)
    # 다른 일자를 기준으로 추출하고 싶은 경우 위 코드 수정 및 주석해제
    
    day_1 = today - datetime.timedelta(1)
    # 전일자 ex> 2022-09-13
    
    start_day = datetime.date(year=day_1.year, month=day_1.month, day=1)
    # 금월 첫 1일자 ex> 2022-09-01
    
    yearmonth = day_1.strftime('%Y%m')
    # 년월 ex> 202209
    
    month_name = str(day_1.month) + '월'
    # 금월 ex> 9월

rdate = report_date()

### 코드
- 전처리 자동화를 위한 코드로 임의 수정을 하실 경우 복사본 생성하여 진행해주시길 바랍니다.
- 혹 아래 이미지와 같은 형식의 오류가 발생할 경우 `!pip install [패키지명]`형식의 코드 작성하여 실행해주시면 해당 패키지가 설치됩니다.
    - <img src = 'https://ifh.cc/g/ymACFT.png' align='left'>

- 이외의 오류가 발생하거나 문제가 해결되지 않는 경우, 데컨팀에 문의주세요 :)

In [ ]:
# 패키지 설치 코드 (패키지 설치 오류 발생시 아래 코드 실행해주세요.)
# !pip install pyarrow
# !pip install datetime

In [ ]:
import os
import pyarrow as pa
import pyarrow.csv as pacsv
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

raw_dir = dr.dropbox_dir + '/광고사업부/4. 광고주/핀다_7팀/2. 리포트/자동화리포트/appsflyer_prism'
result_dir = dr.download_dir
media_source = ['naver_sa_mo_main', '네이버sa', 'naversa', 'kakaosa', 'googlesa', 'naver_sa_mo_direct',
                'naversamo', 'naversapc', 'googlesamo', 'googlesapc', 'googleadwords_int']

def get_raw_df(raw_dir, required_date, media_source):
    dtypes = {
        'attributed_touch_type': pa.string(),
        'attributed_touch_time': pa.string(),
        'install_time': pa.string(),
        'event_time': pa.string(),
        'event_name': pa.string(),
        'event_value': pa.string(),
        'event_revenue': pa.string(),
        'event_revenue_currency': pa.string(),
        'partner': pa.string(),
        'media_source': pa.string(),
        'channel': pa.string(),
        'keywords': pa.string(),
        'campaign': pa.string(),
        'campaign_id': pa.string(),
        'adset': pa.string(),
        'adset_id': pa.string(),
        'ad': pa.string(),
        'ad_id': pa.string(),
        'site_id': pa.string(),
        'sub_site_id': pa.string(),
        'sub_param_1': pa.string(),
        'sub_param_2': pa.string(),
        'sub_param_3': pa.string(),
        'sub_param_4': pa.string(),
        'sub_param_5': pa.string(),
        'appsflyer_id': pa.string(),
        'advertising_id': pa.string(),
        'idfa': pa.string(),
        'customer_user_id': pa.string(),
        'imei': pa.string(),
        'idfv': pa.string(),
        'platform': pa.string(),
        'is_retargeting': pa.string(),
        'is_primary_attribution': pa.string(),
        'original_url': pa.string(),
        'keyword_id': pa.string()
    }
    index_columns = list(dtypes.keys())
    convert_ops = pacsv.ConvertOptions(column_types=dtypes, include_columns=index_columns)
    ro = pacsv.ReadOptions(block_size=10 << 20)
    table_list = []

    date_check = required_date.strftime('%Y%m')
    start_date = required_date.replace(day=1).strftime('%Y%m%d')
    end_date = required_date.strftime('%Y%m%d')

    files = os.listdir(raw_dir)
    files = [f for f in files if '.csv' in f and str(f)[-12:-6] == date_check]
    raw_files = [f for f in files if
                 (int(str(f)[-12:-4]) >= int(start_date)) & (int(str(f)[-12:-4]) <= int(end_date))]

    for f in raw_files:
        temp = pacsv.read_csv(raw_dir + '/' + f, convert_options=convert_ops, read_options=ro)
        table_list.append(temp)
    table = pa.concat_tables(table_list)
    raw_df = table.to_pandas()
    raw_df.media_source = raw_df.media_source.str.lower()
    raw_df = raw_df.loc[raw_df['media_source'].isin(media_source)]
    return raw_df


def prep_data(raw_df):
    prep_df = raw_df
    prep_df[['attributed_touch_time','install_time','event_time']] = prep_df[['attributed_touch_time','install_time','event_time']].apply(pd.to_datetime)
    prep_df['event_date'] = pd.to_datetime(prep_df['event_time']).dt.date

    # install 데이터 가공 :: event_name in ['install','re-attribution','re-engagement']
    prep_df = prep_df.drop(index=prep_df.loc[(prep_df['event_name'].isin(['install','re-attribution','re-engagement']))
                                           &(prep_df['media_source']=='naversamo')&(prep_df['event_date'] == '2022-08-02')].index)
    prep_df = prep_df.drop(index=prep_df.loc[(prep_df['event_name'].isin(['install','re-attribution','re-engagement']))
                                            &(prep_df['media_source']=='googleadwords_int')&(~prep_df['channel'].isin(['Search']))].index)
    prep_df = prep_df.drop(index=prep_df.loc[(prep_df['event_name'].isin(['install','re-attribution','re-engagement']))
                                             &(prep_df['attributed_touch_type']!='click')].index)
    prep_df['CTIT'] = (prep_df['install_time'] - prep_df['attributed_touch_time']).apply(lambda x: x.total_seconds()/(60*60*24))
    prep_df = prep_df.drop(index=prep_df.loc[(prep_df['event_name'].isin(['install','re-attribution','re-engagement']))
                                             &(prep_df['CTIT'] > 7)].index)

    # in-app events 데이터 가공 :: event_name in ['Viewed LA Home','Clicked Signup Completion Button','loan_contract_completed','MD_complete_view']
    prep_df = prep_df.drop(index=prep_df.loc[(prep_df['event_name'].isin(['Viewed LA Home','Clicked Signup Completion Button','loan_contract_completed','MD_complete_view']))
                                           &(prep_df['media_source']=='naversamo')&(prep_df['event_date'] == '2022-08-02')].index)
    prep_df = prep_df.drop(index=prep_df.loc[(prep_df['event_name'].isin(['Viewed LA Home','Clicked Signup Completion Button','loan_contract_completed','MD_complete_view']))
                                             & (prep_df['media_source'] == 'googleadwords_int') & (~prep_df['channel'].isin(['Search']))].index)
    prep_df = prep_df.drop(index=prep_df.loc[(prep_df['event_name'].isin(['Viewed LA Home','Clicked Signup Completion Button','loan_contract_completed','MD_complete_view']))
                                             & (prep_df['attributed_touch_type'] != 'click')].index)
    prep_df = prep_df.drop(
        index=prep_df.loc[(prep_df['event_name'].isin(['Viewed LA Home','Clicked Signup Completion Button','loan_contract_completed','MD_complete_view']))
                          & (prep_df['CTIT'] > 7)].index)
    prep_df['ITET'] = (prep_df['event_time'] - prep_df['install_time']).apply(lambda x: x.total_seconds()/(60*60*24))
    prep_df = prep_df.drop(index=prep_df.loc[(prep_df['event_name'].isin(['Viewed LA Home','Clicked Signup Completion Button','loan_contract_completed','MD_complete_view']))
                                             &(prep_df['ITET'] > 30)].index)

    prep_df.loc[prep_df['keywords'].isin(['', '{keyword}']),'keywords'] = '-'
    prep_df.loc[prep_df['campaign'] == '', 'campaign'] = '-'
    prep_df.loc[prep_df['adset'] == '', 'adset'] = '-'
    prep_df.loc[prep_df['ad'] == '', 'ad'] = '-'

    prep_df.loc[prep_df['ad'] == 'utm_content', 'keywords'] = prep_df['adset']
    prep_df.loc[prep_df['ad'] == 'utm_content', ['adset', 'ad']] = '-'
    prep_df.loc[(prep_df['ad'] == 'bridge_finda-brand-2_2207')&(prep_df['adset'] == '대출'), 'keywords'] = prep_df['adset']
    prep_df.loc[(prep_df['ad'] == 'bridge_finda-brand-2_2207') & (prep_df['adset'] == '대출'), ['adset', 'ad']] = '-'
    # 추가 예외처리

    return prep_df


def download_df(prep_df, required_date, result_dir):
    date = required_date.strftime('%m%d')
    writer = pd.ExcelWriter(result_dir + f'/종합_{date}.xlsx', engine='xlsxwriter', engine_kwargs={'options':{'strings_to_urls': False}})

    # 데이터 구분
    loan_total = prep_df.loc[prep_df['event_name']=='loan_contract_completed'].reset_index(drop=True)
    loan_total.loc[:,'event_name'] = 'loan_contract_completed TOTAL'
    install_total = prep_df.loc[prep_df['event_name'].isin(['install','re-engagement','re-attribution'])].reset_index(drop=True)
    prep_unique = prep_df.sort_values(by='event_time').drop_duplicates(['is_retargeting', 'event_name', 'appsflyer_id'], keep='first')
    event_total = prep_unique.loc[prep_unique['event_name'].isin(['Viewed LA Home','Clicked Signup Completion Button','loan_contract_completed','MD_complete_view'])]
    event_total = pd.concat([event_total, loan_total], axis=0).reset_index(drop=True)

    install_summary = install_total[['attributed_touch_type', 'event_date', 'media_source', 'keywords', 'ad', 'campaign', 'adset', 'is_retargeting', 'event_time']]
    install_summary['is_retargeting'] = install_summary['is_retargeting'].apply(lambda x: 'RE' if x == 'True' else 'UA')
    install_summary = install_summary.rename(columns={'is_retargeting':'ua/re'})
    event_summary = event_total[['attributed_touch_type', 'event_date', 'media_source', 'keywords', 'ad', 'campaign', 'adset', 'event_name', 'is_retargeting', 'event_time', 'attributed_touch_time', 'ITET']]
    event_summary['is_retargeting'] = event_summary['is_retargeting'].apply(lambda x: 'RE' if x == 'True' else 'UA')
    event_summary['차이(date)'] = event_summary['ITET'].astype(np.int)
    event_summary = event_summary.rename(columns={'is_retargeting': 'ua/re','ITET':'차이(time)'})

    # 데이터 출력
    install_summary.to_excel(writer, sheet_name='install(summary)', index=False)
    event_summary.to_excel(writer, sheet_name='event(summary)', index=False)
    install_total.to_excel(writer, sheet_name='install(total)', index=False)
    event_total.to_excel(writer, sheet_name='event(total)', index=False)

    writer.close()
    print('download success')


required_date = rdate.day_1
raw_df = get_raw_df(raw_dir, required_date, media_source)
prep_df = prep_data(raw_df)
download_df(prep_df, required_date, result_dir)

---